Copyright (c) 2024 Houssem Ben Braiek, Emilio Rivera-Landos, IVADO, SEMLA

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# The Modelling Process
In this notebook, we also leverage our library to automate repetitive tasks, ensuring a more efficient and organized workflow.

In [ ]:
#LOAD ENV VARIABLES
import os
from dotenv import load_dotenv
load_dotenv()
MLFLOW_URI = os.getenv("MLFLOW_URI")

In [ ]:
%load_ext autoreload
%autoreload 2
# TODO: comment out if you want to check info logs
# import logging
# logging.basicConfig(level=logging.INFO)
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from bank_marketing.data.load_datasets import load_dataset_from_dvc

In [ ]:
os.environ

### Load the data splits

1. Read dvc info direct **from disk** (using `load_dataset_from_dvc`)
1. Add info harcoded into the dict in this notebook (merge python object)
1. Fetch from dvc with fsspec (update: actually in the function this is done)

In [ ]:
# TODO(PARTICIPANT): Update IF NEEDED the data_dir
dataset, dvc_info = load_dataset_from_dvc(data_dir="../data")

In [ ]:
try:
    import dvc.api
    print(dvc.api.get_url("../data/splits/train_x.csv"))
except Exception:
    pass

In [ ]:
dvc_info

### END - Load the data splits

### Init the constants

In [ ]:
person_info_cols_cat = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'comm_type']
person_info_cols_num = ['age']
num_cols_wo_customer = [
    'curr_n_contact',
    'days_since_last_campaign',
    'last_n_contact',
    'emp.var.rate',
    'cons.price.idx',
    'cons.conf.idx',
    'euribor3m',
    'nr.employed',
]
cat_cols_wo_customer = ['comm_month', 'comm_day', 'last_outcome']
numerical_cols = person_info_cols_num + num_cols_wo_customer
categorical_cols = person_info_cols_cat + cat_cols_wo_customer
predictors = person_info_cols_cat + person_info_cols_num + num_cols_wo_customer + cat_cols_wo_customer
predicted = 'curr_outcome'

#### Creating info dictionaries

Notice we add information about the DVC file:

In [ ]:
classic_ds_info = {
    "numerical_features": numerical_cols,
    "categorical_features": categorical_cols,
    "predicted_target": predicted,
    "dvc_info": dvc_info,
}

In [ ]:
adv_ds_info = {
    "person_info_num_features": person_info_cols_num,
    "person_info_cat_features": person_info_cols_cat,
    "wo_customer_num_features": num_cols_wo_customer,
    "wo_customer_cat_features": cat_cols_wo_customer,
    "predicted_target": predicted,
    "dvc_info": dvc_info,
}

In [ ]:
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
mlflow.set_tracking_uri(MLFLOW_URI)
from bank_marketing.models.skl_tracked_train_models import(
    train_and_evaluate_with_tracking,
    advanced_train_and_evaluate_with_tracking
) 
from bank_marketing.mlflow.tracking import Experiment
from bank_marketing.features.skl_build_features import (
    FeatureNames,
    make_data_transformer,
    AdvFeatureNames,
    make_advanced_data_transformer,
)
from bank_marketing.models.skl_train_models import (
    accuracy_evaluation,
    print_accuracy_results
)
from bank_marketing.mlflow.tracking import get_best_run, explore_best_runs

## Iteration 1
In this initial iteration, we prepare the dataset for binary classification modeling and proceed to test our first baseline model, the Decision Tree, using our library.

In [ ]:
experiment_name = 'decision_tree_tryout'
experiment = Experiment(MLFLOW_URI, experiment_name)
experiment_id = mlflow.set_experiment(experiment.name).experiment_id

In [ ]:
feature_names = FeatureNames(numerical_cols, categorical_cols)

In [ ]:
with mlflow.start_run(experiment_id=experiment_id):
    model = Pipeline(
        [
            ('input_transformer', make_data_transformer(feature_names)),
            ('classifier', DecisionTreeClassifier()),
        ]
    )
    model.fit(dataset.train_x, dataset.train_y)
    results = accuracy_evaluation(model, dataset)
    mlflow.log_metric("train_accuracy", results['train'])
    mlflow.log_metric("valid_accuracy", results['val'])
    mlflow.log_metric("test_accuracy", results['test'])
    print_accuracy_results(results)
    sample = dataset.train_x.sample(5)
    signature = infer_signature(sample, 
                                model.predict(sample))
    mlflow.sklearn.log_model(model, "model", 
                             input_example=sample,
                             signature=signature,
                             extra_pip_requirements=["bank-marketing"])
    mlflow.log_dict(classic_ds_info, "data.yml")
    display = RocCurveDisplay.from_predictions(dataset.val_y.values,
                                            model.predict_proba(dataset.val_x)[:,1])
    mlflow.log_figure(display.figure_, 'plots/ValidRocCurveDisplay.png')
    display = RocCurveDisplay.from_predictions(dataset.test_y.values,
                                            model.predict_proba(dataset.test_x)[:,1])
    mlflow.log_figure(display.figure_, 'plots/TestRocCurveDisplay.png')

## Iteration 2:
The second iteration is about testing multiple models and compare their performance. 

In [ ]:
experiment_name = 'classic_classif_exp'
experiment = Experiment(MLFLOW_URI, experiment_name)
classifiers_list = [DecisionTreeClassifier, MLPClassifier, RandomForestClassifier]
results = train_and_evaluate_with_tracking(dataset, feature_names, classifiers_list, experiment, classic_ds_info)
print_accuracy_results(results)

In [ ]:
explore_best_runs(experiment)

## Iteration 3
The third iteration is about trying out advanced feature engineering with classifier model: Mini-Batch KMeans and Random Forest. 

In [ ]:
experiment_name = 'random_forest_minibatch_kmeans_tryout'
experiment = Experiment(MLFLOW_URI, experiment_name)
experiment_id = mlflow.set_experiment(experiment.name).experiment_id

In [ ]:
adv_feature_names = AdvFeatureNames(
    person_info_cols_num, person_info_cols_cat, num_cols_wo_customer, cat_cols_wo_customer
)

In [ ]:
with mlflow.start_run(experiment_id=experiment_id):
    model = Pipeline(
        [
            ('input_transformer', make_advanced_data_transformer(adv_feature_names, MiniBatchKMeans())), 
            ('classifier', RandomForestClassifier()),
        ]
    )
    model.fit(dataset.train_x, dataset.train_y)
    results = accuracy_evaluation(model, dataset)
    mlflow.log_metric("train_accuracy", results['train'])
    mlflow.log_metric("valid_accuracy", results['val'])
    mlflow.log_metric("test_accuracy", results['test'])
    print_accuracy_results(results)
    sample = dataset.train_x.sample(5)
    signature = infer_signature(sample, 
                                model.predict(sample))
    mlflow.sklearn.log_model(model, "model", 
                             input_example=sample,
                             signature=signature,
                             )
    mlflow.log_dict(adv_ds_info, "data.yml")
    display = RocCurveDisplay.from_predictions(dataset.val_y.values,
                                            model.predict_proba(dataset.val_x)[:,1])
    mlflow.log_figure(display.figure_, 'plots/ValidRocCurveDisplay.png')
    display = RocCurveDisplay.from_predictions(dataset.test_y.values,
                                            model.predict_proba(dataset.test_x)[:,1])
    mlflow.log_figure(display.figure_, 'plots/TestRocCurveDisplay.png')

## Iteration 4
The fourth iteration is about testing advanced feature engineering with multiple models and comparing their performance. 

In [ ]:
experiment_name = 'adv_classifier_exp'
experiment = Experiment(MLFLOW_URI, experiment_name)

In [ ]:
classifiers_list = [DecisionTreeClassifier, RandomForestClassifier]
clustering_algos_list = [KMeans, MiniBatchKMeans]
results = advanced_train_and_evaluate_with_tracking(
    dataset, adv_feature_names, clustering_algos_list, 
    classifiers_list, experiment, adv_ds_info
)
print_accuracy_results(results)

In [ ]:
best_run = get_best_run(experiment)

## Final Step
It is important now to register our best advanced model into model registry. We will show how to transition to staging, as well as promoting to production using bank-marketing library, however, these steps should be done after careful validation.

In [ ]:
import bank_marketing.mlflow.registry as registry

In [ ]:
model_name = "camp-accept-predictor"

In [ ]:
model_version = registry.register_model_from_run(experiment.tracking_server_uri, best_run, model_name)

In [ ]:
registry.transition_model_to_staging(experiment.tracking_server_uri, model_name, model_version.version)

In [ ]:
model_version = registry.promote_model_to_production(experiment.tracking_server_uri, model_name, f"{model_name}-production")

In [ ]:
registry.set_active_production_model(experiment.tracking_server_uri,  f"{model_name}-production", model_version.version)